Capstone Two - Create an instantaneous development summary for newly launched pre-sale multi-family residential projects at launch. Consisting of: 
a) merge with price details from "Price List" (Table 2, denoted as pl, with number indicator for subsequent price lists).
b) merge information with the "Sales Arrangement" (Table 3, denoted as sa).
c) merge "Registration of Transaction" (Table 4, denoted as rot) data, including ASP, P-ASP and Cancelled transactions.

With presentation of a summary table of the sales progress. This can be extended for application in newly launched projects within the same area and launch timing; and further extended for secondary market transactions. The challenge is the files are all in pdf format. The price list tables are updated as news of the pre-sale is to be launched. 

"Nice to hav items" for next step development:
a) Prices based on maximum price discounts offered by the payment plans. Or categories based on payment plan type.
b) a image analysis tool to categorise floor plan images (pdf format) based on room types from "Sales Brochure" (Table 1, denoted as sb).
c) flat details including area (GFA, NSA, balconies, floor, tower, view etc.) from "Sales Brochure" (Table 1, denoted as sb).



In [2]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import pandas_profiling
from pandas_profiling.utils.cache import cache_file
import tabula
import re
import camelot

In [3]:
# Import data
# Data saved locally from website (only available in pdf format).
# As a test case, we will use the recently launched presale project "One Victoria".

# Import the Tabula-py module (written in Java) which reads the the PDF document and converts the Python DataFrame 
# into JSON object.
#---------------------------------------------------------------------------------------------------------------------#

# Price List (price data)
pdf_pl1 = 'OneVictoriaPL120210624Final.pdf'
tabula.convert_into(pdf_pl1, "raw_pl1.csv", output_format="csv", pages='2-23', lattice = True)

pdf_pl2 = 'OneVictoriaPL220210628FINAL.pdf'
#tabula.convert_into(pdf_pl2, "raw_pl2.csv", output_format="csv", pages='2-12', lattice = True)
# Problematic table price list 2
tab_pl2 = tabula.read_pdf(pdf_pl2, pages='2-12', lattice = True)

pdf_pl3 = 'OneVictoriaPL320210629FINAL.pdf'
tabula.convert_into(pdf_pl3, "raw_pl3.csv", output_format="csv", pages='2-13', lattice = True)

pdf_pl4 = 'OneVictoriaPL420210704Final.pdf'
tabula.convert_into(pdf_pl4, "raw_pl4.csv", output_format="csv", pages='2-12', lattice = True)

pdf_pl5 = 'OneVictoriaPL520210706Final.pdf'
tabula.convert_into(pdf_pl5, "raw_pl5.csv", output_format="csv", pages='2-16', lattice = True)

#---------------------------------------------------------------------------------------------------------------------#

# Sales Arrangements (sales launch date data)
pdf_sa1 = 'OV Sales Arrangements 20210629.pdf'
tabula.convert_into(pdf_sa1, 'raw_sa1.csv',output_format="csv", pages='2', lattice=True)

pdf_sa2 = 'OV Sales Arrangements 20210706.pdf'
tabula.convert_into(pdf_sa2, 'raw_sa2.csv',output_format="csv", pages='2', lattice=True)

#---------------------------------------------------------------------------------------------------------------------#

# Register of Transactions (transaction record and sales progress) 
# NOTE: this is regularly updated and the link may need checking from time to time.
pdf_rot = 'One Victoria_RT 20211023 2045.pdf'
#tabula.read_pdf(pdf_rot, pages='2', area=(180,10,600,480), lattice = True)
tabula.convert_into(pdf_rot, "raw_rot.csv", output_format="csv", pages='2-502', area=(180,10,800,800), lattice = True)

# Areial photograph of the development is also available, however a google view of the location is better 
# for view categorisation (building height should also be taken into account).

# Furthermore, news reports phrases on the sales sentiment and actual progress can also be explored. However, a lot of 
# these news reports will be in Chinese.


In [4]:
# Define development inputs
Development_Name = 'One_Vic'

In [5]:
# Problematic table price list 2, clean and export as csv
tbl_0 = tab_pl2[0].iloc[1:10,0:6]
tbl_0.columns = ['Tower', 'Floor','Unit', 'Area', 'Price', 'Price/SA']

tbl_temp = tab_pl2[2].iloc[:,[1,3,5,7,9,11]].dropna() 

for i in range(4,22,2):
    tbl_temp = tbl_temp.append(tab_pl2[i].iloc[:,[1,3,5,7,9,11]].dropna() )
tbl_temp

tbl_temp.columns = ['Tower', 'Floor','Unit', 'Area', 'Price', 'Price/SA']

pl2 = pd.concat([tbl_0, tbl_temp], axis=0)
pl2.reset_index(drop=True, inplace=True)

# Area in sq.ft.
pl2_Area_temp = pl2["Area"].str.split(r"\(", expand=True).iloc[:,1]
pl2_Area = pl2_Area_temp.str.split(r"\)", expand=True).iloc[:,0]
        
# Format Price/SA
pl2_pricesa_temp = pl2["Price/SA"].str.split(r"\(", expand=True).iloc[:,1]
pl2_pricesa = pl2_pricesa_temp.str.split(")", expand=True).iloc[:,0]
    
# Format tower
pl2_tower = pl2['Tower'].str.split(r"\r", expand=True).iloc[:,0]

# Combine tables
pl2_final = pd.concat([pl2_tower, pl2['Floor'], pl2['Unit'], pl2_Area, pl2['Price'], pl2_pricesa], axis=1)
pl2_final.columns = ['Tower', 'Floor','Unit', 'Area', 'Price', 'Price/SA']

# Add in unique ID
pl2_final['PL_no'] = 'PL2'
pl2_final['ID'] = Development_Name+pl2_final['Tower']+pl2_final['Floor']+pl2_final['Unit']


In [6]:
def pl_import(n_csv, pl_no):
    """Import price list csv and clean data."""
    pl_rcsv = pd.read_csv(n_csv)

    # correct data, remove duplicates, rename columns
    pl_temp = pl_rcsv.iloc[1:,0:6]
    pl_df = pl_temp[~pl_temp.duplicated()]

    pl_df.columns = ['Tower', 'Floor','Unit', 'Area', 'Price', 'Price/SA']

    # Area in sq.ft.
    pl_Area_temp = pl_df["Area"].str.split(r"\(", expand=True).iloc[:,1]
    pl_Area = pl_Area_temp.str.split(r"\)", expand=True).iloc[:,0]
        
    # Format Price/SA
    pl_pricesa_temp = pl_df["Price/SA"].str.split(r"\(", expand=True).iloc[:,1]
    pl_pricesa = pl_pricesa_temp.str.split(")", expand=True).iloc[:,0]
    
    # Format tower
    pl_tower = pl_df['Tower'].str.split(r"\r", expand=True).iloc[:,0]

    # Combine tables
    pl_final = pd.concat([pl_tower, pl_df['Floor'], pl_df['Unit'], pl_Area, pl_df['Price'], pl_pricesa], axis=1)
    pl_final.columns = ['Tower', 'Floor','Unit', 'Area', 'Price', 'Price/SA']

    # Add in unique ID
    pl_final['PL_no'] = pl_no
    pl_final['ID'] = Development_Name+pl_final['Tower']+pl_final['Floor']+pl_final['Unit']
    
    # Remove weird characters
    pl_final = pl_final[~pl_final.Tower.str.contains(r'[?@#&$%+-/*]')]
        
    return pl_final


In [7]:
# Import price lists into master price list
pl1 = pl_import('raw_pl1.csv', 'PL1')
pl3 = pl_import('raw_pl3.csv', 'PL3')
pl4 = pl_import('raw_pl4.csv', 'PL4')
pl5 = pl_import('raw_pl5.csv', 'PL5')

pl = pl1.append([pl2_final, pl3, pl4, pl5])
pl.reset_index(drop=True, inplace=True)
pl
pl.to_csv('PL.csv')

In [8]:
# Clean the ROT data
rot_df = pd.read_csv("raw_rot.csv")
rot_df = rot_df.iloc[:,[0,1,3,4,5,7,9]]
rot_df.columns = ['Date_PASP', 'Date_ASP', 'Tower', 'Floor', 'Unit', 'Trans_Price','Payment_Terms']
rot_df['Date_PASP'] = rot_df['Date_PASP'].str.replace("-","/")
rot_df['Date_ASP'] = rot_df['Date_PASP'].str.replace("-","/")
rot_df['Tower'] = rot_df['Tower'].str.split(r"\r", expand=True).iloc[:,0]

rot_df['ID'] = Development_Name+rot_df['Tower']+rot_df['Floor'].map(str)+rot_df['Unit']

# Format Payment Terms
rot_df['Payment_Terms'] = rot_df['Payment_Terms'].str.split(r'[()]', expand=True).iloc[:,1]

#rot_pay_terms
rot_df
rot_df.to_csv("ROT.csv")

In [9]:
def sa_extract(csv_file, first_sale_date):
    sa_df = pd.read_csv(csv_file, header = None)
    sa_df = sa_df[0].str.split(r"\r|,", expand=True)

    sa = sa_df.transpose()
    sa = sa[~sa[0].str.contains(r'[?]')]
    sa[sa[0]==""] = np.NaN

    # Make new column for tower code
    sa = sa[0].str.split(("Tower"), expand=True)
    sa[1] = sa[1].str.split(r"(", expand=True)
    sa[sa[1]==""] = np.NaN

    # Fill down rows Tower code
    sa[1] = sa[1].fillna(method='ffill')

    # Keep only valid data, remove empty rows
    sa = sa[sa[0].apply(lambda x: len(str(x)) <= 5)]
    sa.dropna(how='any', inplace=True)
    sa[0] = sa[0].str.replace(" ","")
    sa[1] = sa[1].str.replace(" ","")

    # Set ID code
    sa['first_sale'] = first_sale_date
    sa['ID'] = Development_Name+"Tower "+sa[1]+sa[0]

    sa.drop([0,1], inplace=True, axis=1)
    
    return sa

In [10]:
# Extract data from SA1, first sale date = '03/07/2021' (DD/MM/YYYY)
sa1 = sa_extract('raw_sa1.csv', '03/07/2021')
sa1.to_csv('SA1.csv')

# Extract data from SA2, first sale date = '10/07/2021' (DD/MM/YYYY)
sa2 = sa_extract('raw_sa2.csv', '10/07/2021')
sa2.to_csv('SA2.csv')


In [36]:
# Create master table

flatmix_df = pd.read_excel("Flat Mix.xlsx")
rot_temp = rot_df[['ID', 'Date_PASP', 'Date_ASP', 'Trans_Price', 'Payment_Terms']]
pl_temp = pl[['Price', 'Price/SA','ID']]
master1 = pd.merge(flatmix_df, pl_temp, how='left', on='ID')
master2 = pd.merge(master1, rot_temp, how='left', on="ID")
master3 = pd.merge(master2, sa1, how='left', on="ID")
master = pd.merge(master3, sa2, how='left', on=["ID", "first_sale"])
master.to_csv("master.csv")
master

,ID,Tower,Floor,Unit,Area,Flat Type,No. of Rooms,Price,Price/SA,Date_PASP,Date_ASP,Trans_Price,Payment_Terms,first_sale
0,One_VicTower 1A1A,Tower 1A,1,A,643,3BR + 2B + 1K,3BR,"16,137,000","25,096",03/07/2021,03/07/2021,"$13,958,000",A,03/07/2021
1,One_VicTower 1A1B,Tower 1A,1,B,444,2BR + 1B + 1OK,2BR,"10,613,000","23,903",03/07/2021,03/07/2021,"$9,604,000",C,03/07/2021
2,One_VicTower 1A1C,Tower 1A,1,C,438,2BR + 1B + 1OK,2BR,"10,739,000","24,518",10/07/2021,10/07/2021,"$9,718,000",C,NaN
3,One_VicTower 1A1D,Tower 1A,1,D,444,2BR + 1B + 1OK,2BR,"11,355,000","25,574",03/07/2021,03/07/2021,"$9,822,000",A,03/07/2021
4,One_VicTower 1A1E,Tower 1A,1,E,426,2BR + 1B + 1OK,2BR,"11,009,000","25,843",03/07/2021,03/07/2021,"$9,963,000",C,03/07/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,One_VicTower 2B36D,Tower 2B,36,D,457,2BR + 1B + 1OK,2BR,"14,703,000","32,173",NaN,NaN,NaN,NaN,NaN
1045,One_VicTower 2B36E,Tower 2B,36,E,452,2BR + 1B + 1OK,2BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1046,One_VicTower 2B36F,Tower 2B,36,F,1360,4BR + 3B + +1K + 1S + 1T + 1FR,4BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1047,One_VicTower 2B36J,Tower 2B,36,J,355,1BR + 1B + 1OK,1BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
